##### **This notebook creates the coordination graph for each user**

In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config

In [ ]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']
plot_path = path['plot_path']

file_hp.create_folder(derived_path, 'user_coordination_graph')
reply_path = os.path.join(derived_path, 
                          'user_coordination_graph')

# file_hp.create_folder(reply_path, campaign)
# coord_path = os.path.join(reply_path, campaign)

In [3]:
# campaign = 'iran_201901_1'
# year = '2019_01'
# type_of = 'ops'
#
#1686
year = '2020_05'
type_of = 'ops'
campaign = 'china_052020'

In [4]:
file_hp.create_folder(reply_path, campaign)
coord_path = os.path.join(reply_path, campaign)

In [5]:
def convert_tweetids_to_string(df_time, field='tweetid',
                               filter_threshold=False,
                               threshold=None
                              ) -> pd.DataFrame:
    '''
    Converts the time binned tweets to single single dataframe and convert the tweets \
    to single document tweets for each users
    
    :param df_time: dictionary of time binned dataframes
    
    :return pandas dataframe
    '''

    df_time = df_time.astype({field: int})
    df_time = df_time.astype({field: str})
    
    df_time = (df_time
                   .groupby('userid')[field]
                   .apply(list)
                   .reset_index(name='tweet_ids'))
        
    if filter_threshold is not False:
        df_time = df_time.loc[df_time['tweet_ids'].map(len) > threshold]
    
    df_time['tweet_ids'] = df_time['tweet_ids'].apply(lambda x: ' '.join(x))
    
    return df_time

In [6]:
def coordination_reply(df, userid, 
                       output_path):
    # print('\n----- Start: Filtering tweets ---------')
    # # df = filter_reply_count(df, 5)
    # print('----- End: Filtering tweets ---------\n')
    # print(df['in_reply_to_tweetid']
    # print('\n----- Start: Create bi-partite network -----')
    df_network = hp.create_user_projection(df, 
                                        'in_reply_to_tweetid')
    # print('\n----- End: Create bi-partite network -----')
    
    # print('\n----- Start: Create co-reply strings  -----')
    df_string = convert_tweetids_to_string(df, 
                                           field='in_reply_to_tweetid')
    # print('\n----- End: Create co-reply strings -----')
    
    # print('\n----- Start: Calculate tfidf vector -----')
    df_string = hp.calculate_tf_idf_vector(df_string)
    # print('\n------ End: Calculate tfidf vector -----')
    
    # print('\n----- Start: Retweet user projection network ------')
    df_new = hp.calculate_cosine_similarity(df_network, 
                                            df_string)
    # print('\n----- End: Retweet user projection network -----')
       
    pkl_path = os.path.join(output_path, 
                            f'{userid}_user_network.pkl.gz')
    
    df_new.to_pickle(f'{pkl_path}')
    
    # print('\n ----- Start: Create co-retweet graph -----')
    # create_graph(df_new, 
    #              output_path,
    #              campaign_name=campaign_name,
    #              source_column='source',
    #              target_column='target',
    #              weight_column='cosine',
    #              type_text='co-reply')
    # print(f'----- End: Creating user projection network ---------\n')
        
    return df_new

In [7]:
def create_user_graph(df):
    '''
    Creates reply coordination graph for user
    
    :param df: dataframe
    :param
    '''
    
    pass

In [8]:
def prepare_data(year, campaign):
    data_path = st.get_data_path(all_tweet_data, 
                                 year, 
                                 campaign)

    data = st.read_ops_control_data(data_path['ops'],
                                    data_path['control'], 
                                 [type_of])
    df_w = data[type_of]
    df = df_w.loc[~df_w['in_reply_to_tweetid'].isnull()]
    
    df = st.reply_to_external_users(df)

    ### only users which 
    df_users = (df.groupby(['in_reply_to_userid'])[['userid',
                                 'in_reply_to_tweetid']]
              .size()
              .to_frame('count')
              .reset_index())
    groups = df_users.loc[df_users['count'].map(len) > 0]
    groups = df.groupby(['in_reply_to_userid'])[['userid',
                                 'in_reply_to_tweetid']]
    
    print(len(groups))
    print(coord_path)
    
    for grp in groups:
        df_user = grp[1].reset_index(drop=True)
        
        name = str(int(grp[0]))
        df_coord = coordination_reply(df_user, 
                                      name, 
                                      coord_path)

In [9]:
prepare_data(year, campaign)

18883
/N/slate/potem/data/derived/user_coordination_graph/china_052020
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userid_x  0 non-null      int64
 1   userid_y  0 non-null      int64
dtypes: int64(2)
memory usage: 0.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userid     1 non-null      int64 
 1   tweet_ids  1 non-null      object
 2   vector     1 non-null      object
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userid_x  0 non-null      int64 
 1   userid_y  0 non-null      int64 
 2   userid    0 non-null      int64 
 3   vector

100%|██████████| 2/2 [00:00<00:00, 26886.56it/s]

[1.]

 after filtering 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   userid_x  2 non-null      int64  
 1   userid_y  2 non-null      int64  
 2   cosine    2 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 64.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userid_x  0 non-null      int64
 1   userid_y  0 non-null      int64
dtypes: int64(2)
memory usage: 0.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userid     1 non-null      int64 
 1   tweet_ids  1 non-null      object
 2   vector     1 non-null      object
dtypes: int64(1), object(2)
memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userid     1 non-null      int64 
 1   tweet_ids  1 non-null      object
 2   vector     1 non-null      object
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userid_x  0 non-null      int64 
 1   userid_y  0 non-null      int64 
 2   userid    0 non-null      int64 
 3   vector    0 non-null      object
dtypes: int64(3), object(1)
memory usage: 0.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userid_x  0 non-null      int64
 1   userid_y  0 non-null      int64
dtypes: int64(2)
memory usage: 0

100%|██████████| 124/124 [00:00<00:00, 1281018.96it/s]


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userid     1 non-null      int64 
 1   tweet_ids  1 non-null      object
 2   vector     1 non-null      object
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userid_x  0 non-null      int64 
 1   userid_y  0 non-null      int64 
 2   userid    0 non-null      int64 
 3   vector    0 non-null      object
dtypes: int64(3), object(1)
memory usage: 0.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 1 to 14
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userid_x  12 non-null     int64
 1   userid_y  12 non-null     int64
dtypes: int64(2)
memor

100%|██████████| 12/12 [00:00<00:00, 186413.51it/s]

[1.]

 after filtering 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   userid_x  12 non-null     int64  
 1   userid_y  12 non-null     int64  
 2   cosine    12 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 384.0 bytes
None


<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   userid_x  0 non-null      int64
 1   userid_y  0 non-null      int64
dtypes: int64(2)
memory usage: 0.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userid     1 non-null      int64 
 1   tweet_ids  1 non-null      object
 2   vector     1 non-null      object
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userid_x  0 non-null      int64 
 1   userid_y  0 non-null      int64 
 2   userid    0 non-null      int64 
 3   vector    0 non-null      object
dtypes: int64(3), object(1)
memory usage: 0.


KeyboardInterrupt



In [ ]:
print(coord_path)